<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 9:** LangServe 和評量(Assessment)</font>

<br>

## LangServe 伺服器設置


這個 notebook 是為那些有興趣使用 LangChain 和 [**LangServe**](https://python.langchain.com/docs/langserve) 開發互動式網路應用程式的人提供的練習場。目標是提供一個最小程式碼範例來說明 LangChain 在網路應用程式脈絡資訊(Context)中的潛力。

本節提供了使用 LangChain 的 Runnable 介面與 FastAPI 設置簡單 API 伺服器的演練。範例展示了如何整合 LangChain 模型，如 `ChatNVIDIA`，來創建和分發可存取的 API 路由(Routes)。使用這個，您將能夠為前端服務的 [**`frontend_server.py`**](https://www.perplexity.ai/search/frontend/frontend_server.py) 會話提供功能，該會話強烈期望：

-   一個名為 `:9012/basic_chat` 的簡單端點(Endpoints)用於基本聊天機器人，如下所示。

-   一對名為 `:9012/retriever` 和 `:9012/generator` 的端點(Endpoints)用於 RAG 聊天機器人。

-   所有三個用於**評估**實用工具，這將是最終評量(Assessment)所需的。*稍後詳述！*

**重要注意事項：**

-   確保點擊方形 ( $\square$ ) 按鈕兩次來關閉啟動中的 FastAPI 程式碼區塊。第一次可能會失敗或在非同步過程上觸發 try-catch 例程。

-   如果仍然無法運作，請使用 **Kernel -> Restart Kernel** 對此 notebook 進行硬重啟。

-   當 FastAPI 伺服器在您的程式碼區塊中運行時，預期該過程會阻擋此 notebook。其他 notebook 不應受此影響。

<br>

### **第一部分：** 提供 /basic_chat 端點(Endpoints)


提供了作為獨立 Python 檔案啟動 `/basic_chat` 端點(Endpoints)的指令。這將被前端用來進行沒有內部分析推理(Reasoning)的基本決策。

In [3]:
%%writefile server_app.py
# https://python.langchain.com/docs/langserve#server
from fastapi import FastAPI
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langserve import add_routes

## May be useful later
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables import RunnableLambda, RunnableBranch, RunnablePassthrough
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_community.document_transformers import LongContextReorder
from functools import partial
from operator import itemgetter

from langchain_community.vectorstores import FAISS

## TODO: Make sure to pick your LLM and do your prompt engineering as necessary for the final assessment
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
instruct_llm = ChatNVIDIA(model="meta/llama3-8b-instruct")

app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple api server using Langchain's Runnable interfaces",
)

## PRE-ASSESSMENT: Run as-is and see the basic chain in action

add_routes(
    app,
    instruct_llm,
    path="/basic_chat",
)

## ASSESSMENT TODO: Implement these components as appropriate

add_routes(
    app,
    RunnableLambda(lambda x: "Not Implemented"),
    path="/generator",
)

add_routes(
    app,
    RunnableLambda(lambda x: []),
    path="/retriever",
)

## Might be encountered if this were for a standalone python file...
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=9012)

Overwriting server_app.py


In [ ]:
## Works, but will block the notebook.
!python server_app.py  

## Will technically work, but not recommended in a notebook. 
## You may be surprised at the interesting side effects...
# import os
# os.system("python server_app.py &")

INFO:     Started server process [449]
INFO:     Waiting for application startup.

     __          ___      .__   __.   _______      _______. _______ .______     ____    ____  _______
    |  |        /   \     |  \ |  |  /  _____|    /       ||   ____||   _  \    \   \  /   / |   ____|
    |  |       /  ^  \    |   \|  | |  |  __     |   (----`|  |__   |  |_)  |    \   \/   /  |  |__
    |  |      /  /_\  \   |  . `  | |  | |_ |     \   \    |   __|  |      /      \      /   |   __|
    |  `----./  _____  \  |  |\   | |  |__| | .----)   |   |  |____ |  |\  \----.  \    /    |  |____
    |_______/__/     \__\ |__| \__|  \______| |_______/    |_______|| _| `._____|   \__/     |_______|
    
LANGSERVE: Playground for chain "/basic_chat/" is live at:
LANGSERVE:  │
LANGSERVE:  └──> /basic_chat/playground/
LANGSERVE:
LANGSERVE: Playground for chain "/generator/" is live at:
LANGSERVE:  │
LANGSERVE:  └──> /generator/playground/
LANGSERVE:
LANGSERVE: Playground for chain "/retriever/" is live

<br>

### **第二部分：** 使用伺服器：


雖然這在 Google Colab 中無法輕易使用（或至少不是沒有很多特殊技巧），但上述腳本將保持一個與 notebook 過程綁定的運行伺服器。當伺服器運行時，不要嘗試使用此 notebook（除了關閉/重啟服務）。

然而，在另一個檔案中，您應該能夠使用以下介面存取 `basic_chat` 端點(Endpoints)：


```python
from langserve import RemoteRunnable
from langchain_core.output_parsers import StrOutputParser

llm = RemoteRunnable("http://0.0.0.0:9012/basic_chat/") | StrOutputParser()
for token in llm.stream("Hello World! How is it going?"):
    print(token, end='')
```

**請在不同的檔案中試用並看看是否有效！**


<br>

### **第三部分：最終評量(Assessment)**


**此 notebook 將用於完成最終評量(Assessment)！** 當您完成課程的其他部分時，我們建議複製此 notebook，在新標籤中打開前端，並透過實作上述的 `/generator` 和 `/retriever` 端點(Endpoints)來實作評估功能！要快速連結到前端，請運行下面的程式碼區塊：


In [ ]:
%%js
var url = 'http://'+window.location.host+':8090';
element.innerHTML = '<a style="color:#76b900;" target="_blank" href='+url+'><h2>< Link To Gradio Frontend ></h2></a>';

<hr>
<br>

#### **評量(Assessment)提示：**


請注意以下功能已在前端微服務中實作。


```python
## Necessary Endpoints
chains_dict = {
    'basic' : RemoteRunnable("http://lab:9012/basic_chat/"),
    'retriever' : RemoteRunnable("http://lab:9012/retriever/"),  ## For the final assessment
    'generator' : RemoteRunnable("http://lab:9012/generator/"),  ## For the final assessment
}

basic_chain = chains_dict['basic']

## Retrieval-Augmented Generation Chain

retrieval_chain = (
    {'input' : (lambda x: x)}
    | RunnableAssign(
        {'context' : itemgetter('input') 
        | chains_dict['retriever'] 
        | LongContextReorder().transform_documents
        | docs2str
    })
)

output_chain = RunnableAssign({"output" : chains_dict['generator'] }) | output_puller
rag_chain = retrieval_chain | output_chain
```

**為了符合此端點(Endpoints)輸入(Intake)策略，請確保不要重複管線(Pipeline)功能，只部署架構(Deployment)缺失的功能！**

----

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>